# Imports

In [2]:
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Reading data

In [3]:
df = pd.read_csv('dataset/trainning-tweets.csv',encoding = "ISO-8859-1",names=['sentiment','id','date','Query','user','tweet']) 

In [4]:
X = df.tweet.values.tolist()
y = df.sentiment.values.tolist()

# Caracterização dos textos com TFIDF e remoção de stopwords

In [ ]:
vect = TfidfVectorizer(stop_words='english')

In [7]:
mnb = MultinomialNB(alpha=0.5)
# log_model = LogisticRegression()
# stemmer = nltk.stem.RSLPStemmer()


In [8]:
pipe = Pipeline(steps=[('vect', vect)
                       ,('mnb', mnb)
                      ])
param_grid = {
}

In [9]:
search = GridSearchCV(pipe, param_grid, iid=False, cv=5, return_train_score=False)
search.fit(X, y)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...rue,
        vocabulary=None)), ('mnb', MultinomialNB(alpha=0.5, class_prior=None, fit_prior=True))]),
       fit_params=None, iid=False, n_jobs=None, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
       scoring=None, verbose=0)

In [10]:
result_df = pd.DataFrame(search.cv_results_)
result_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,19.361239,0.623962,3.980518,0.191145,{},0.746156,0.737269,0.7337,0.748744,0.740966,0.741367,0.005533,1
